In [ ]:
import os,glob
import datetime
import jaydebeapi
import pandas as pd
import subprocess
from hdfs import InsecureClient
from hdfs import Client
from absl import flags,app
from absl.flags import FLAGS

### 하둡연결

In [ ]:
client_hdfs = InsecureClient('http://10.183.134.15:50070', user='bdip')

In [ ]:
print(client_hdfs.list('/user/bdip/data/smart/input'))

### 하둡 다운로드

In [ ]:
flags.DEFINE_string('FROM', '/user/bdip/data/smart/input/','hadoop download directory path')
flags.DEFINE_string('TO', '/data/open_dt/','local directory path to download')
flags.DEFINE_string('todo', '2021102817','dir name todo')
flags.DEFINE_string('f','','kernel')

In [ ]:
def main(_argv):
    c = Clinet('http://10.183.134.15:50070',proxy='bdip')
    
    home_dir = FLAGS.TO
    output_dir = os.path.join(home_dir, 'download')
    
    todo_dir = FLAGS.todo
    hadoop_dir = FLAGS.FROM
    
    if todo_dir not in c.list(hadoop_dir):
        print(f'{todo_dir} not in hadoop server')
        
        return
    
    download_path = os.path.join(hadoop_dir, todo_dir)
    save_dir_path = os.path.join(output_dir, todo_dir)
    os.makedirs(save_dir_path, exist_ok = True)
    print(f'download {todo_dir}\n')
    
    
    # Fetch data from Cubrid
    conn = jaydebeapi.connect('cubrid.jdbc.driver.CUBRIDriver', 
                             'jdbc:cubrid:10.188.208.65:30000:BDIP_POTL_DB:::?charset=utf-8',
                             ['bdip','Bdip123#'],
                             '/data/open_df/JDBC-10.2-latest-cubrid.jar')
    
    curr = conn.cursor()
    
    todo_timestamp = datetime.datetime.strptime(todo_dir,'%Y%m%d%H')
    next_timestamp = todo_timestamp+datetime.timedelta(hours=1)
    
    sql = f"""
            SELECT * 
            FROM itfc_smt_tpof_outpt
            WHERE load_dt BETWEEN '{todo_timestamp}' AND '{next_timestamp}'
        """    
    
    curr.execute(sql)
    out_data = curr.fetchall()
    columns = [i[0] for i in curr.description]
    
    df = pd.DataFrame(data=out_data,columns=columns)
    curr.close()
    
    size_dict = {f'{i}_{j}':k for i,j,k in zip(df.trop_sn,df.tpof_data_sn,df.file_size)}
    
    
    # download start
    total_todo = c.list(download_path)
    now_save_dir = os.listdir(save_dir_path)
    
    count = 0
    
    
    for todo in total_todo:
        download_file_path = os.path.join(download_path,todo)
        save_file_path = os.path.join(save_dir_path,todo)
        
        tpof_sn = todo.split('_')[0]
        tpof_data_sn = todo.split('_')[1]
        
        c.download(download_file_path, save_file_path, overwrite=True)
        
        # download file check
        if os.path.isfile(save_file_path) and (os.path.getsize(save_file_path) == size_dict[f'{tpof_sn}_{tpof_data_sn}']):
            count +=1
        else:
            print(f'[error] {save_file_path}')
            ind = [(df.tpof_sn == tpof_sn) & (df.tpof_data_sn = tpof_data_sn)].index[0]
            df = df.drop(index=ind)
            
       
        os.makedirs(os.path.join(home_dir, f'cubrid/{todo_dir}'), exist_ok = True
        df.to_csv(os.path.join(home_dir,f'cubrid/{todo_dir}/itfc_smt_tpof_outpt.csv', encoding = 'utf-8'), index=False)
        print(f"finish download {count}")
        print(f"[{datetime.datetime.now()}]")
                    
                    
if __name__ = "__main__":
    try:
        app.run(main)
    except SystemExit:
        pass            
        
    
    